In [2]:
import os 
import pandas as pd
import numpy as np
from ast import literal_eval
from collections import Counter
from operator import itemgetter

## 1. Read in data/drop columns to reduce size

In [56]:
DATA_DIR = os.getenv("DATA_DIR")
REPORTS_DIR = os.getenv("REPORTS_DIR")
filename = "dlo_prelim_meta_standard_with_pageseq_from_29-10_to_04-11-2018.csv.gz"
# filename = "brexit_taxon_29_04.csv.gz"
# filename = "reduced_preprocessed_taxon_pageseq_20190114_20190116.csv.gz"
df_file = os.path.join(DATA_DIR, "processed_journey", filename)
df_reduced_file = os.path.join(DATA_DIR, "processed_journey", filename)
# df_rel_file = os.path.join(DATA_DIR, "processed_journey", "rel_"+filename)

### Load up unfiltered data: 14-16/01/19, one off run

Original dataset shape (5048130, 17)

In [57]:
# df = pd.read_csv(df_file, sep="\t", compression = "gzip")
# print(df.shape)
# df.drop(['Dates', 'Page_List', 'Event_List', 'Taxon_List', 'Taxon_Page_List', 'Page_List_NL'],axis=1,inplace=True)
# df.to_csv(df_reduced_file, sep="\t", compression="gzip", index=False)

## 2. User journey analysis

In [58]:
# sep="\t",
df = pd.read_csv(df_reduced_file, compression="gzip")

In [59]:
## Total volume of traffic
df.Occurrences.sum()

6537680

In [60]:
def column_eval(cols):
    for column in cols:
        if not isinstance(df[column].iloc[0],list):
            print(column)
            df[column] = df[column].map(literal_eval)

In [61]:
# 'Page_Event_List', 
columns = ['DeviceCategories', 'Event_cats_agg', 'Event_cat_act_agg']
column_eval(columns)

DeviceCategories
Event_cats_agg
Event_cat_act_agg


#### For older data cuts that do not include `PageSeq_Length`

In [105]:
if "PageSeq_Length" not in df.columns:
    df['Page_List'] = df['Page_List'].map(literal_eval)
    df['PageSeq_Length'] = df['Page_List'].map(len)

### 2.1 Out of all journeys, what is the % (volume, sum of Occurrences) of journeys including "related content" link clicks?

In [62]:
n_rel = df[(df.Sequence.str.contains("Related content")) & (df.Sequence.str.contains("relatedLinkClicked"))].shape[0]

In [63]:
# 473975
"{}% of journeys that contain a \"related link\" click".format(round((n_rel*100)/df.Occurrences.sum(),2))

'5.67% of journeys that contain a "related link" click'

Run this at some point to compare against "related content" link-specific dataset. 
There seems to be an indication that there are no major differences

### 2.2 Out of all journeys coming from mobiles vs desktops, what is the % (volume, sum of Occurrences) of journeys including "related content" link clicks?

In [64]:
def more_device(x,device):
    return max(x,key=itemgetter(1))[0]==device 
def device_count(x,device):
    return sum([value for item, value in x if item==device])      

In [65]:
df["DesktopCount"] = df['DeviceCategories'].map(lambda x: device_count(x,"desktop"))
df["MobileCount"] = df['DeviceCategories'].map(lambda x: device_count(x,"mobile"))        

In [66]:
df["TabletCount"] = df['DeviceCategories'].map(lambda x: device_count(x,"tablet"))        

In [67]:
df["DesktopCount"].describe()

count    3.788851e+06
mean     9.399570e-01
std      7.376457e+01
min      0.000000e+00
25%      0.000000e+00
50%      1.000000e+00
75%      1.000000e+00
max      1.307680e+05
Name: DesktopCount, dtype: float64

### 2.3 Traffic volume
#### Test implementation

In [68]:
related = df[(df.Sequence.str.contains("Related content")) & (df.Sequence.str.contains("relatedLinkClicked"))].iloc[0]

In [69]:
related

Sequence                /british-citizenship/after-your-citizenship-ce...
Occurrences                                                             1
DeviceCategories                                            [(mobile, 1)]
Dates                                                   [('20181031', 1)]
Page_Event_List         [('/british-citizenship/after-your-citizenship...
Page_List               ['/british-citizenship/after-your-citizenship-...
PageSequence            /british-citizenship/after-your-citizenship-ce...
Event_List              [('PAGE_NULL', 'PAGE_NULL'), ('relatedLinkClic...
num_event_cats                                                          2
Event_cats_agg                  [(PAGE_NULL, 2), (relatedLinkClicked, 1)]
Event_cat_act_agg       [((PAGE_NULL, PAGE_NULL), 2), ((relatedLinkCli...
Page_List_NL            ['/british-citizenship/after-your-citizenship-...
Page_Seq_NL             /british-citizenship/after-your-citizenship-ce...
Page_Seq_Occurrences                  

In [70]:
explore = df[~(df.Sequence.str.contains("Related content")) & (df.Sequence.str.contains("relatedLinkClicked"))].iloc[0]

In [71]:
explore

Sequence                /employment-tribunal-decisions/miss-s-ward-v-d...
Occurrences                                                             1
DeviceCategories                                            [(mobile, 1)]
Dates                                                   [('20181031', 1)]
Page_Event_List         [('/employment-tribunal-decisions/miss-s-ward-...
Page_List               ['/employment-tribunal-decisions/miss-s-ward-v...
PageSequence            /employment-tribunal-decisions/miss-s-ward-v-d...
Event_List              [('PAGE_NULL', 'PAGE_NULL'), ('relatedLinkClic...
num_event_cats                                                          2
Event_cats_agg                  [(PAGE_NULL, 2), (relatedLinkClicked, 1)]
Event_cat_act_agg       [((PAGE_NULL, PAGE_NULL), 2), ((relatedLinkCli...
Page_List_NL            ['/employment-tribunal-decisions/miss-s-ward-v...
Page_Seq_NL             /employment-tribunal-decisions/miss-s-ward-v-d...
Page_Seq_Occurrences                  

In [72]:
explore.Event_cat_act_agg

[(('PAGE_NULL', 'PAGE_NULL'), 2), (('relatedLinkClicked', '1'), 1)]

In [73]:
#Compute whether a journey includes a related link click
def is_related(x):
    return all(cond in x for cond in ["relatedLinkClicked","Related content"])

In [74]:
print(is_related(related.Sequence))
print(is_related(df.Sequence.iloc[0]))
print(is_related(explore))

True
False
False


In [75]:
df["Has_Related"] = df["Sequence"].map(is_related)

In [76]:
df[df["Has_Related"]].Occurrences.sum()

452404

### 2.4 Journeys per device
Counter({'desktop': 256791, 'tablet': 55546, 'mobile': 161638})

In [135]:
index = ["All","All_related","Desktop","Desktop_rel","Mobile","Mobile_rel"]
columns = ["Volume", "Percentage", "Shape"]
df_stats = pd.DataFrame(index=index,columns=columns)

,Volume,Percentage,Shape
All,NaN,NaN,NaN
All_related,NaN,NaN,NaN
Desktop,NaN,NaN,NaN
Desktop_rel,NaN,NaN,NaN
Mobile,NaN,NaN,NaN
Mobile_rel,NaN,NaN,NaN


### Compute all volumes 

In [159]:
def compute_all_stats(df, desktop_journeys, filtering, df_stats):
    vol_all = df.Occurrences.sum()
    vol_all_related = df[df.Has_Related].Occurrences.sum()
    ## Number of journeys coming from desktops
    vol_desk = df["DesktopCount"].sum()
    ## Number of journeys coming from mobiles
    vol_mobile = df["MobileCount"].sum()
    ## Number of journeys coming from tablets
    vol_tablet = df["TabletCount"].sum()

    ## Seperate out desktop and mobile journeys
    desktop_journeys = df[df.DesktopCount>0]
    mobile_journeys = df[df.MobileCount>0]

    desk_rel_journeys = desktop_journeys[desktop_journeys.Has_Related]
    mobile_rel_journeys = mobile_journeys[mobile_journeys.Has_Related]

    ## Compute number of journeys from specific device that include related links
    ## Don't base counting on occurrences, will include excluded device
    vol_desk_rel = desktop_journeys[desktop_journeys.Has_Related].DesktopCount.sum()
    vol_mobile_rel = mobile_journeys[mobile_journeys.Has_Related].MobileCount.sum()
    
    vols = [vol_all, vol_all_related,
        vol_desk, vol_desk_rel,
        vol_mobile, vol_mobile_rel]
    
    percent_related = round((vol_all_related*100)/vol_all,2)
    percent_from_desk = round((vol_desk*100)/df.Occurrences.sum(),2)
    percent_from_mobile = round((vol_mobile*100)/df.Occurrences.sum(),2)
    percent_from_tablet = round((vol_tablet*100)/df.Occurrences.sum(),2)
    percent_from_desk_rel = round((vol_desk_rel*100)/vol_desk,2)
    percent_from_mobile_rel = round((vol_mobile_rel*100)/vol_mobile,2)
    
    percents = [100, percent_related,
            percent_from_desk, percent_from_desk_rel,
            percent_from_mobile, percent_from_mobile_rel]
    
    shape_all = df.shape[0]
    shape_all_rel = df[df.Has_Related].shape[0]
    shape_desk = desktop_journeys.shape[0]
    shape_desk_rel = desktop_journeys[desktop_journeys.Has_Related].shape[0]
    shape_mobile = mobile_journeys.shape[0]
    shape_mobile_rel = mobile_journeys[mobile_journeys.Has_Related].shape[0]
   
    shapes = [shape_all, shape_all_rel,
         shape_desk, shape_desk_rel,
         shape_mobile, shape_mobile_rel]
    
    df_stats["Volume"] = vols
    df_stats["Percentage"] = percents
    df_stats["Shape"] = shapes
    
    df_stats.to_csv(os.path.join(REPORTS_DIR,filename.replace(".csv.gz",filtering)+"_stats.csv"))
    
    print("{}% of journeys come from a desktop".format(percent_from_desk))
    print("{}% of journeys come from a mobile".format(percent_from_mobile))
    print("{}% of journeys come from a tablet".format(percent_from_tablet))
    print("{}% of desktop journeys include a \"Related content\" click".format(percent_from_desk_rel))
    print("{}% of mobile journeys include a \"Related content\" click".format(percent_from_mobile_rel))

In [160]:
compute_all_stats(df,"",df_stats)
df_stats

54.47% of journeys come from a desktop
36.65% of journeys come from a mobile
8.88% of journeys come from a tablet
6.51% of desktop journeys include a "Related content" click


NameError: name 'percent_from_desk_tab' is not defined

In [161]:
df_stats

,Volume,Percentage,Shape
All,6537680,100.00,3788851
All_related,452404,6.92,370366
Desktop,3561357,54.47,2245236
Desktop_rel,231793,6.51,191942
Mobile,2395941,36.65,1318885
Mobile_rel,162580,6.79,136958


In [ ]:
compute_all_stats(df[df.PageSeq_Length>1], "_dlo",df_stats)
df_stats

### 2.5 Chi-squared test
Compute observations for a contingency table

In [84]:
from scipy import stats

In [187]:
vol_desk = df_stats.loc['Desktop','Volume']
vol_mobile = df_stats.loc['Mobile','Volume']

vol_mobile_rel = df_stats.loc['Mobile_rel','Volume']
vol_mobile_no_rel = vol_mobile - vol_mobile_rel

vol_desk_rel = df_stats.loc['Desktop_rel','Volume']
vol_desk_no_rel = vol_desk - vol_desk_rel

In [245]:
def chi2_test(obs):
    chi2, p, dof, ex = stats.chi2_contingency(obs)
    print(chi2, p, dof, ex)
    
#     g, p, dof, expctd = stats.chi2_contingency(obs, lambda_="log-likelihood")

    with open(os.path.join(REPORTS_DIR, filename.replace(".csv.gz","")+"_chi2.csv"),"w") as f:
        f.write("chi2,p,dof\n")
        f.write("{},{},{}".format(chi2, p, dof))

In [246]:
obs = [[vol_mobile_rel, vol_mobile_no_rel], [vol_desk_rel, vol_desk_no_rel]]
obs

[[162580, 2233361], [231793, 3329564]]

In [247]:
chi2_test(obs)

177.8466031986581 1.4309008012162503e-40 1 [[ 158611.24288108 2237329.75711892]
 [ 235761.75711892 3325595.24288108]]


In [248]:
ch = pd.read_csv(os.path.join(REPORTS_DIR, filename.replace(".csv.gz","")+"_chi2.csv"))
ch

,chi2,p,dof
0,177.846603,1.430901e-40,1


### 2.6 User journey length

In [96]:
def weight_seq_length(page_lengths, occurrences, name):
    length_occ = Counter()
    for length,occ in zip(page_lengths, occurrences):
           length_occ[length]+=occ
    data = []
    for key,value in length_occ.items():
        for i in range(value):
            data.append(key)
    return pd.Series(data,name=name)

#### Everything together

In [101]:
def describe_dfs(to_eval):
    descriptive = pd.DataFrame()
    for length,occ,name in to_eval:
        sr = weight_seq_length(length,occ,name).describe().apply(lambda x: format(x, '.3f'))
        descriptive[sr.name] = sr
    return descriptive

In [114]:
list_of_cols = [[df.PageSeq_Length, df.Occurrences, "All_Journeys"],
                [df[df.Has_Related].PageSeq_Length, df[df.Has_Related].Occurrences, "All_Journeys_Related"],
                [desktop_journeys.PageSeq_Length, desktop_journeys.DesktopCount,"All_Desktop"],
                [mobile_journeys.PageSeq_Length, mobile_journeys.MobileCount, "All_Mobile"],
                [desk_rel_journeys.PageSeq_Length, desk_rel_journeys.DesktopCount, "Desktop_Related"],
                [mobile_rel_journeys.PageSeq_Length, mobile_rel_journeys.MobileCount, "Mobile_Related"]]

In [122]:
describe = describe_dfs(list_of_cols)
describe.to_csv(os.path.join(REPORTS_DIR,filename.replace(".csv.gz","")+"_pageseq_length_all.csv"),index=False)

In [121]:
describe

,All_Journeys,All_Journeys_Related,All_Desktop,All_Mobile,Desktop_Related,Mobile_Related
count,6537680.000,452404.000,3561357.000,2395941.000,231793.000,162580.000
mean,4.823,8.953,4.978,4.548,9.176,8.642
std,5.950,9.862,6.581,4.908,10.590,8.888
min,0.000,2.000,0.000,1.000,2.000,2.000
25%,2.000,3.000,2.000,2.000,3.000,3.000
50%,3.000,6.000,3.000,3.000,6.000,6.000
75%,5.000,11.000,5.000,5.000,11.000,11.000
max,500.000,444.000,500.000,358.000,444.000,318.000


#### Exclude user journeys of length 1

In [ ]:
df_dlo = df[df.PageSeq_Length > 1]
## Seperate out desktop and mobile journeys
desktop_journeys_dlo = df_dlo[df_dlo.DesktopCount>0].copy(deep=True)
mobile_journeys_dlo =  df_dlo[df_dlo.MobileCount>0].copy(deep=True)
desk_rel_journeys_dlo = desktop_journeys_dlo[desktop_journeys_dlo.Has_Related]
mobile_rel_journeys_dlo = mobile_journeys_dlo[mobile_journeys_dlo.Has_Related]

In [ ]:
list_of_columns = [[df_dlo.PageSeq_Length, df_dlo.Occurrences, "All_Journeys"],
                   [df_dlo.PageSeq_Length, df_dlo.Occurrences, "All_Journeys_Related"],
                   [desktop_journeys_dlo.PageSeq_Length, desktop_journeys_dlo.DesktopCount,"All_Desktop"],
                   [mobile_journeys_dlo.PageSeq_Length, mobile_journeys_dlo.MobileCount, "All_Mobile"],
                   [desk_rel_journeys_dlo.PageSeq_Length, desk_rel_journeys_dlo.DesktopCount, "Desktop_Related"],
                   [mobile_rel_journeys_dlo.PageSeq_Length, mobile_rel_journeys_dlo.MobileCount, "Mobile_Related"]]

In [ ]:
describe_dlo = describe_dfs(list_of_cols)
describe_dlo.to_csv(os.path.join(REPORTS_DIR,filename.replace(".csv.gz","")+"_pageseq_length_dlo.csv"),index=False)

### 2.7 Event frequencies overall

In [ ]:
df.columns

In [ ]:
event_cats = Counter()
for event_agg in df.Event_cats_agg:
    for event,value in event_agg:
        event_cats[event] += value
len(event_cats.keys())

In [ ]:
for key,value in event_cats.items():
    print(key,":",value)

In [ ]:
# def mem_usage(pandas_obj):
#     if isinstance(pandas_obj,pd.DataFrame):
#         usage_b = pandas_obj.memory_usage(deep=True).sum()
#     else: # we assume if not a df it's a series
#         usage_b = pandas_obj.memory_usage(deep=True)
#     usage_mb = usage_b / 1024 ** 2 # convert bytes to megabytes
#     return "{:03.2f} MB".format(usage_mb)

# gl_int = df.select_dtypes(include=['int'])
# converted_int = gl_int.apply(pd.to_numeric,downcast='unsigned')

# print(mem_usage(gl_int))
# print(mem_usage(converted_int))

# compare_ints = pd.concat([gl_int.dtypes,converted_int.dtypes],axis=1)
# compare_ints.columns = ['before','after']
# compare_ints.apply(pd.Series.value_counts)